In [1]:
import pandas as pd
import numpy as np
import string
import json
import re
import time
import io

In [2]:
df = pd.read_csv("ENG_scrambled_.csv")

In [3]:
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
import datetime

def date_clean(df):
    date = parser.parse(df)
#     new_date = date + pd.DateOffset(years=7, months=4)
    date = date.strftime("%Y-%m-%dT%H:%M:%SZ")
    
    return date

In [4]:
df['receivedDateTime '] = df['receivedDateTime '].apply(date_clean)

In [5]:
df.columns

Index(['conversationId ', 'fromEmailAddress ', 'toRecipients ', 'subject ',
       'body.content ', 'receivedDateTime ', 'ccRecipients ', 'id ',
       'commentBy', 'comment', '_allow_permissions', 'type', 'from_user',
       'recipient_user'],
      dtype='object')

In [6]:
df['comment'].replace('NaN', np.NaN)

0                                                      NaN
1        i also get this in 1.6.0 with systematic varia...
2        michael,\n\nif you are willing to hack up your...
3                       i'm experiencing this with 2.1.3rc
4        pr pending personal://github.com/appcelerator/...
                               ...                        
11611    i cannot reproduce the initial issue, [~nunoco...
11612    issue reproduced on both sdk version tokens ga...
11613                                                  NaN
11614    i have this issue, is hard to fix? let me know...
11615    please fix it at 3.2.3 version!\nsame as issue...
Name: comment, Length: 11616, dtype: object

In [7]:
df.dropna(inplace=True)

In [8]:
df.head()

,conversationId,fromEmailAddress,toRecipients,subject,body.content,receivedDateTime,ccRecipients,id,commentBy,comment,_allow_permissions,type,from_user,recipient_user
1,60016,['christopher.franke@gmail.com'],['richard.ellis@example.com'],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"['liam.aylward@example.com', 'christopher.fran...",AAMKALJYLoGJQpGiRqXmqbjeuHAfrCnlMBlZBnfbAbonqc...,['christopher.franke@gmail.com'],i also get this in 1.6.0 with systematic varia...,"['manojpraveen445@gmail.com', 'christopher.fra...",email-message,Christopher Franke,Richard Ellis
2,60016,['christopher.franke@gmail.com'],['richard.ellis@example.com'],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"['liam.aylward@example.com', 'christopher.fran...",AAMKApbePeTmSuCChwhQUENHHXNbetmzPEaiYANaDElQeL...,['laurent.beaudoin@gmail.com'],"michael,\n\nif you are willing to hack up your...","['manojpraveen445@gmail.com', 'christopher.fra...",email-message,Christopher Franke,Richard Ellis
3,60016,['christopher.franke@gmail.com'],['richard.ellis@example.com'],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"['liam.aylward@example.com', 'christopher.fran...",AAMKADdThFgcKsogapXsgTaLemanknIISsXqsfIrRIJIGf...,['lu.shanglei@gmail.com'],i'm experiencing this with 2.1.3rc,"['manojpraveen445@gmail.com', 'christopher.fra...",email-message,Christopher Franke,Richard Ellis
4,60016,['christopher.franke@gmail.com'],['richard.ellis@example.com'],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"['liam.aylward@example.com', 'christopher.fran...",AAMKAIVHcvnXXiVcjQxbKxpcJaatXnNQbbWwQeXkVjjzbP...,['rhonda.glenn@example.com'],pr pending personal://github.com/appcelerator/...,"['manojpraveen445@gmail.com', 'christopher.fra...",email-message,Christopher Franke,Richard Ellis
5,60016,['christopher.franke@gmail.com'],['richard.ellis@example.com'],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"['liam.aylward@example.com', 'christopher.fran...",AAMKACGbgoQFaeYHAANbDTLbkAoEqOsLbnjedXBYCHprEm...,['manojpraveen445@gmail.com'],http://technologies.appcelerator.com/question/...,"['manojpraveen445@gmail.com', 'christopher.fra...",email-message,Christopher Franke,Richard Ellis


In [9]:
with open("final_fake_transient.json", 'r') as f:
    final_fake_tr = json.loads(f.read())
f.close()

In [10]:
df.reset_index(drop=True, inplace=True)

In [11]:
import random

In [12]:
fake_id_df = pd.read_csv('fake_users_with_ids.csv')

In [13]:
fake_id_df.columns

Index(['userId', 'emailId', 'displayName', 'firstName', 'lastName', 'team'], dtype='object')

In [14]:
def user_tag(df):
    return 'USER_' + df

In [15]:
fake_ids = fake_id_df[['emailId', 'userId']].copy()

In [16]:
# fake_ids['userId'] = fake_ids['userId'].apply(user_tag)

In [17]:
fake_tr = dict(zip(list(fake_ids['emailId']), list(fake_ids['userId'])))

In [18]:
fake_tr

{'chris.hughes@example.com': 'a34e9bde-e434-49b4-b6ee-1108af2b7d83',
 'james.kynge@example.com': '19acc115-f0a6-4fe7-9c8f-7037e3ba64f2',
 'tommy.ivo@example.com': '96ec5a12-8d0d-4c7f-b62c-21590dee2496',
 'rhonda.glenn@example.com': '8dd3acad-f6aa-4f69-808b-a35a4a3aeec5',
 'jerry.hunter@example.com': '7d521171-420b-4895-a383-a5cb293699a4',
 'robert.langdon@example.com': 'e0f78954-fabb-4faf-852e-de8e0e646ec2',
 'avner.the@example.com': 'c0454eb6-8508-4afe-979d-e0104f07d51a',
 'yang.ji@example.com': '6a007777-9392-4f85-8eda-68242e3b5f11',
 'milton.moon@example.com': '81270d54-9a4d-42f0-9232-20a1603d8f32',
 'petr.chylek@example.com': '0505431b-c8d4-41ea-a5d3-b6455a43c043',
 'david.clayton@example.com': 'b1db5d1c-8c07-4015-b16d-d4a2d6973f14',
 'arwad.al@example.com': '3477d5fe-3a59-40c2-ab50-dfdb75cfdc01',
 'syd.fischer@example.com': '02a6ca9c-292c-4023-8d2a-4d842fa4db9d',
 'alan.reid@example.com': '1ed0480f-3dae-4ee3-9dc5-a0855b3a36d7',
 'tomasz.urbanowicz@example.com': '3353ec98-fc3d-4f58

In [19]:
# cog_id = []
# for i in range(len(final_fake_tr)):
#     cog_id.append('USER_' + ''.join(random.choices(string.ascii_letters + string.digits, k=36)))

In [20]:
# cog_dict = dict(zip(list(final_fake_tr.keys()), cog_id))

In [21]:
with open('ENG_cog_dict.json', 'r') as f:
    cog_dict = json.loads(f.read())

In [22]:
len(cog_dict)

1016

In [23]:
"hugh.cavendish@example.com" in cog_dict

True

In [24]:
cog_dict

{'chris.hughes@example.com': 'hWUxqJmmxtyFkjCAhmJyOz4bEQrp1NkB9rhC',
 'james.kynge@example.com': 'ADpe2jLopOSub0Vah8HyNv4HMt2hZ084SGwv',
 'tommy.ivo@example.com': 'gOt7T3y6VkqerjpPdULitOyRKNk991kSAMqf',
 'rhonda.glenn@example.com': 'kuFLbGg14JKlbcNynWkzv2SPfKHbBjkNbVvd',
 'jerry.hunter@example.com': 'x2M4lZN5Vl89I2q1FRF4JloBvgTt8Zko04EJ',
 'robert.langdon@example.com': 'c2vcPocE2sRvK3drjsqMzW9yxy9cDn8Px6fg',
 'avner.the@example.com': 'A4xkZqotrhun9Ib8ExXDOgFBkBmiD4MlaANT',
 'yang.ji@example.com': 'oI7zABUThR2kUX6x4HsJVgqV2O2Y5ZmQcR6d',
 'milton.moon@example.com': 'sQ7rNJYkt9jZdESvajOlBDT15j5WtJo2lGeQ',
 'petr.chylek@example.com': 'Y3hQRLKphTWBlkEb5r5RM9NFZJZmxpnNpQ5p',
 'david.clayton@example.com': '5s2KSNR2mMmyebJoxI3qqnr3JZXG0Jt7ARfQ',
 'arwad.al@example.com': '4Z999QsBii2DekVBYKI1oXnUqyG1FZKa4lUt',
 'syd.fischer@example.com': 'qcQ5SoMuXmpE8QvGATaoIXzNg1HUepuuqebO',
 'alan.reid@example.com': 'Hlh7XxyvAvf6jTjxd6NC74B5KDa1yj6qqbab',
 'tomasz.urbanowicz@example.com': 'YzJCQYhEd6rIddnBnu

In [25]:
emp = [w for w in cog_dict if w.endswith('example.com')]

In [26]:
len(emp), len(fake_tr)

(74, 80)

In [27]:
[w for w in emp if w not in fake_tr]

['stone.cold@example.com']

In [28]:
'timothy.price@example.com' in cog_dict

True

In [29]:
[w for w in cog_dict if 'timothy' in w]

['timothy.price@example.com', 'w.timothy@gmail.com']

In [30]:
cog_dict.update(fake_tr)

In [31]:
df.isna().sum()

conversationId        0
fromEmailAddress      0
toRecipients          0
subject               0
body.content          0
receivedDateTime      0
ccRecipients          0
id                    0
commentBy             0
comment               0
_allow_permissions    0
type                  0
from_user             0
recipient_user        0
dtype: int64

In [32]:
# df.replace("Mike Matheny", "Rohit Gandhe", inplace=True)
# df.replace("['mike.matheny@example.com']", "['rohit@augmentor.ai']", inplace=True)

# df.replace("['christian.timm@example.com']", "['manojpraveen445@gmail.com']", inplace=True)
# df.replace("Christian Timm", "Manoj Praveen", inplace=True)

# df.replace("['joy.macphail@example.com']", "['frank@augmentor.ai']", inplace=True)
# df.replace("Joy Macphail", "Frank Preve", inplace=True)

# df.replace("['marilyn.denis@example.com']", "['lazarjankovic030@gmail.com']", inplace=True)
# df.replace("Marilyn Denis", "Lazar Jankovic", inplace=True)

# df.replace("['morten.knutsen@example.com']", "['tarun@augmentor.ai']", inplace=True)
# df.replace("Morten Knutsen", "Tarun Dewan", inplace=True)

# df.replace("['john.stanley@example.com']", "['ben@augmentor.ai']", inplace=True)
# df.replace("John Stanley", "Ben Davies", inplace=True)

# df.replace("['jennifer.morla@example.com']", "['jonathan@augmentor.ai']", inplace=True)
# df.replace("Marilyn Denis", "Jonathan Berry", inplace=True)

In [33]:
df['comment'] = df['comment'].replace("Mike Matheny", "Rohit Gandhe")
df['comment'] = df['comment'].replace("mike matheny", "Rohit Gandhe")
df['comment'] = df['comment'].replace("Christian Timm", "Manoj Praveen")
df['comment'] = df['comment'].replace("christian timm", "Manoj Praveen")
df['comment'] = df['comment'].replace("Joy Macphail", "Frank Preve")
df['comment'] = df['comment'].replace("joy macphail", "Frank Preve")
df['comment'] = df['comment'].replace("Marilyn Denis", "Lazar Jankovic")
df['comment'] = df['comment'].replace("marilyn denis", "Lazar Jankovic")
df['comment'] = df['comment'].replace("Morten Knutsen", "Tarun Dewan")
df['comment'] = df['comment'].replace("morten knutsen", "Tarun Dewan")
df['comment'] = df['comment'].replace("John Stanley", "Ben Davies")
df['comment'] = df['comment'].replace("john stanley", "Ben Davies")
df['comment'] = df['comment'].replace("Marilyn Denis", "Jonathan Berry")
df['comment'] = df['comment'].replace("marilyn denis", "Jonathan Berry")

In [34]:
df.columns

Index(['conversationId ', 'fromEmailAddress ', 'toRecipients ', 'subject ',
       'body.content ', 'receivedDateTime ', 'ccRecipients ', 'id ',
       'commentBy', 'comment', '_allow_permissions', 'type', 'from_user',
       'recipient_user'],
      dtype='object')

In [35]:
# df['_allow_permissions'] = df['_allow_permissions'].str.replace('mike.matheny@example.com', 'rohit@augmentor.ai')
# df['ccRecipients '] = df['ccRecipients '].str.replace('mike.matheny@example.com', 'rohit@augmentor.ai')

# df['_allow_permissions'] = df['_allow_permissions'].str.replace('christian.timm@example.com', 'manojpraveen445@gmail.com')
# df['ccRecipients '] = df['ccRecipients '].str.replace('christian.timm@example.com', 'manojpraveen445@gmail.com')

# df['_allow_permissions'] = df['_allow_permissions'].str.replace('joy.macphail@example.com', 'frank@augmentor.ai')
# df['ccRecipients '] = df['ccRecipients '].str.replace('joy.macphail@example.com', 'frank@augmentor.ai')

# df['_allow_permissions'] = df['_allow_permissions'].str.replace('marilyn.denis@example.com', 'lazarjankovic030@gmail.com')
# df['ccRecipients '] = df['ccRecipients '].str.replace('marilyn.denis@example.com', 'lazarjankovic030@gmail.com')

# df['_allow_permissions'] = df['_allow_permissions'].str.replace('morten.knutsen@example.com', 'tarun@augmentor.ai')
# df['ccRecipients '] = df['ccRecipients '].str.replace('morten.knutsen@example.com', 'tarun@augmentor.ai')

# df['_allow_permissions'] = df['_allow_permissions'].str.replace('john.stanley@example.com', 'ben@augmentor.ai')
# df['ccRecipients '] = df['ccRecipients '].str.replace('john.stanley@example.com', 'ben@augmentor.ai')

# df['_allow_permissions'] = df['_allow_permissions'].str.replace('jennifer.morla@example.com', 'jonathan@augmentor.ai')
# df['ccRecipients '] = df['ccRecipients '].str.replace('jennifer.morla@example.com', 'jonathan@augmentor.ai')


In [36]:
# df['_allow_permissions'] = df['_allow_permissions'].str.replace('jennifer.morla@example.com', 'jonathan@augmentor.ai')

In [37]:
# df[df['_allow_permissions'].str.contains("hugh.cavendish@example.com")]

In [38]:
# df.to_csv('ENG_scrambled_.csv', index=False)

In [39]:
eval(df['fromEmailAddress '].loc[0]), eval(df['ccRecipients '].loc[0])[0]

(['christopher.franke@gmail.com'], 'liam.aylward@example.com')

In [40]:
[w for w in eval(df['ccRecipients '].loc[0]) if w!= eval(df['fromEmailAddress '].loc[0])[0]]

['liam.aylward@example.com',
 'laurent.beaudoin@gmail.com',
 'lu.shanglei@gmail.com',
 'rhonda.glenn@example.com',
 'manojpraveen445@gmail.com']

In [41]:
def wrangle_recp():
    resp = []
    for i in range(len(df)):
        ccs = [w for w in eval(df['ccRecipients '].loc[i]) if w not in eval(df['fromEmailAddress '].loc[i])[0]]
        try:
            resp.append(eval(df['toRecipients '].loc[i]) + ccs)
        except Exception as e:
            print(i)
            pass
    return resp

In [42]:
count_df = df.groupby(by='conversationId ').count()
count_df = pd.DataFrame(count_df['commentBy'])

In [43]:
df['recipients'] = wrangle_recp()

In [44]:
df['toRecipients '].loc[0]

"['richard.ellis@example.com']"

In [45]:
df = df[['conversationId ', 'fromEmailAddress ', 'subject ',
       'body.content ', 'receivedDateTime ', 'id ', '_allow_permissions', 'from_user', 'recipients']].copy()

In [46]:
un_df = df.drop_duplicates(subset='conversationId ', keep="first")

In [47]:
un_df.head()

,conversationId,fromEmailAddress,subject,body.content,receivedDateTime,id,_allow_permissions,from_user,recipients
0,60016,['christopher.franke@gmail.com'],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,AAMKALJYLoGJQpGiRqXmqbjeuHAfrCnlMBlZBnfbAbonqc...,"['manojpraveen445@gmail.com', 'christopher.fra...",Christopher Franke,"[richard.ellis@example.com, liam.aylward@examp..."
5,60648,['huey.dunbar@example.com'],via tender: newby - error when launching nodes...,"assigned to <a href=\n""http://support.appceler...",2018-08-14T00:00:00Z,AAMKAzgVFaUsZiloXsOlouOwjckesQDriUijOXuZRvwUdo...,"['cyndy.violette@example.com', 'simon.cawkwell...",Huey Dunbar,"[tommy.ivo@example.com, tommy.ivo@example.com,..."
10,60659,['robert.langdon@example.com'],services - reseexampleh connect integration,description\nthe purpose of this services is t...,2018-08-14T00:00:00Z,AAMKAYIJtPMTNywhyyXdWPCcTWjPPClYHaDonwloarWLnM...,"['robert.langdon@example.com', 'peter.st@examp...",Robert Langdon,"[peter.st@example.com, huey.dunbar@example.com..."
15,60677,['peter.miller@gmail.com'],"navbar color on ""more"" tab",my tiapp.xml file contains about 7 windows. al...,2018-08-14T00:00:00Z,AAMKAIidqMCjMLkQBcsfHCOjOOKIqSOwXJhBJBDqMUtVcP...,"['oliver.wilson@example.com', 'timothy.price@e...",Peter Miller,"[oliver.wilson@example.com, peter.st@example.c..."
20,60757,['denis.ducharme@gmail.com'],notationss are too limited,i've been tinkering around with ti to see if i...,2018-08-14T00:00:00Z,AAMKArAFMDpNrkxnVzCeSVqmLjoRhIIfnoPKizcEDVufsu...,"['daniel.kopans@example.com', 'tommy.ivo@examp...",Denis Ducharme,"[oliver.wilson@example.com, al.franken@example..."


In [48]:
# un_df = un_df.copy()

In [49]:
un_df.reset_index(drop=True, inplace=True)

In [50]:
un_df = un_df.copy()

In [51]:
un_df['type'] = "email-message"

In [52]:
un_df['recipients'].loc[1]

['tommy.ivo@example.com',
 'tommy.ivo@example.com',
 'tommy.ivo@example.com',
 'tommy.ivo@example.com',
 'cyndy.violette@example.com',
 'cyndy.violette@example.com',
 'simon.cawkwell@example.com']

In [53]:
[w for w in final_fake_tr.keys() if w.startswith('alexander')]

['alexander.viktorovich@gmail.com',
 'alexander.schalck@gmail.com',
 'alexander.beliavsky@gmail.com',
 'alexander.lloyd@gmail.com',
 'alexander.tetelbaum@gmail.com']

In [54]:
def user_dict(df):
    res = {}
    user_id = []
    user_display_name = []
    email = []

    for i in df:
        user_id.append(cog_dict[i])
        user_display_name.append(final_fake_tr[i])
        email.append(i)
    res['user_id'] = user_id
    res['user_display_name'] = user_display_name
    res['email'] = email
    res_df = pd.DataFrame.from_dict(res)
    return json.loads(json.dumps(res_df.to_dict('records')))
        

In [55]:
user_dict(['manojpraveen445@gmail.com'])

[{'user_id': '1f4c0330-32c9-49dd-8e25-4fc6347f9703',
  'user_display_name': 'Manoj Praveen',
  'email': 'manojpraveen445@gmail.com'}]

In [56]:
def add_permissions(df):
    add_p = []
    for i in df:
        add_p.append(i)
        if i in fake_tr:
            add_p.append(cog_dict[i])
    return add_p

In [57]:
def evaluator(df):
    df = eval(df)
    df = list(pd.Series(df).unique())
    return df

In [58]:
un_df['_allow_permissions'] = un_df['_allow_permissions'].apply(evaluator)

In [59]:
un_df['_allow_permissions'] = un_df['_allow_permissions'].apply(add_permissions)

In [60]:
un_df['recipients_'] = un_df['recipients'].apply(user_dict)

In [61]:
un_df['fromEmailAddress '] = un_df['fromEmailAddress '].apply(evaluator)

In [62]:
un_df['from_'] = un_df['fromEmailAddress '].apply(user_dict)

In [63]:
un_df['from_name'] = un_df['from_user'].copy()

In [64]:
un_df.columns

Index(['conversationId ', 'fromEmailAddress ', 'subject ', 'body.content ',
       'receivedDateTime ', 'id ', '_allow_permissions', 'from_user',
       'recipients', 'type', 'recipients_', 'from_', 'from_name'],
      dtype='object')

In [65]:
un_df = un_df[['conversationId ', 'subject ', 'body.content ',
       'receivedDateTime ', 'id ', '_allow_permissions',
       'recipients_', 'type', 'from_']].copy()

In [66]:
# un_df['id '] = un_df['conversationId '].copy()

In [67]:
un_df.head()

,conversationId,subject,body.content,receivedDateTime,id,_allow_permissions,recipients_,type,from_
0,60016,iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,AAMKALJYLoGJQpGiRqXmqbjeuHAfrCnlMBlZBnfbAbonqc...,"[manojpraveen445@gmail.com, 1f4c0330-32c9-49dd...",[{'user_id': '374b2ff7-020f-4571-959c-b0a38267...,email-message,"[{'user_id': 'christopher.franke@gmail.com', '..."
1,60648,via tender: newby - error when launching nodes...,"assigned to <a href=\n""http://support.appceler...",2018-08-14T00:00:00Z,AAMKAzgVFaUsZiloXsOlouOwjckesQDriUijOXuZRvwUdo...,"[cyndy.violette@example.com, 3e7bca78-5ee0-4a7...",[{'user_id': '96ec5a12-8d0d-4c7f-b62c-21590dee...,email-message,[{'user_id': 'b570965f-414c-4868-9dc2-ae2beb50...
2,60659,services - reseexampleh connect integration,description\nthe purpose of this services is t...,2018-08-14T00:00:00Z,AAMKAYIJtPMTNywhyyXdWPCcTWjPPClYHaDonwloarWLnM...,"[robert.langdon@example.com, e0f78954-fabb-4fa...",[{'user_id': '2f46bee1-e642-4850-a307-a4bdaec0...,email-message,[{'user_id': 'e0f78954-fabb-4faf-852e-de8e0e64...
3,60677,"navbar color on ""more"" tab",my tiapp.xml file contains about 7 windows. al...,2018-08-14T00:00:00Z,AAMKAIidqMCjMLkQBcsfHCOjOOKIqSOwXJhBJBDqMUtVcP...,"[oliver.wilson@example.com, 575901f1-ff79-485a...",[{'user_id': '575901f1-ff79-485a-86a3-c9104954...,email-message,"[{'user_id': 'peter.miller@gmail.com', 'user_d..."
4,60757,notationss are too limited,i've been tinkering around with ti to see if i...,2018-08-14T00:00:00Z,AAMKArAFMDpNrkxnVzCeSVqmLjoRhIIfnoPKizcEDVufsu...,"[daniel.kopans@example.com, 96b3cc5a-b028-4222...",[{'user_id': '575901f1-ff79-485a-86a3-c9104954...,email-message,"[{'user_id': 'denis.ducharme@gmail.com', 'user..."


In [68]:
count_df.reset_index(inplace=True)
count_df.columns
count_df.columns = ['conversationId ', 'num_messages']
un_df = un_df.merge(count_df, on='conversationId ')

In [69]:
un_df.columns = ['conversation_id', 'subject ', 'body',
       'date', 'id', '_allow_permissions', 'recipients', 'type',
       'from', 'num_messages']

In [70]:
un_df['id'] = un_df['conversation_id']

In [71]:
un_df.head()

,conversation_id,subject,body,date,id,_allow_permissions,recipients,type,from,num_messages
0,60016,iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,60016,"[manojpraveen445@gmail.com, 1f4c0330-32c9-49dd...",[{'user_id': '374b2ff7-020f-4571-959c-b0a38267...,email-message,"[{'user_id': 'christopher.franke@gmail.com', '...",5
1,60648,via tender: newby - error when launching nodes...,"assigned to <a href=\n""http://support.appceler...",2018-08-14T00:00:00Z,60648,"[cyndy.violette@example.com, 3e7bca78-5ee0-4a7...",[{'user_id': '96ec5a12-8d0d-4c7f-b62c-21590dee...,email-message,[{'user_id': 'b570965f-414c-4868-9dc2-ae2beb50...,5
2,60659,services - reseexampleh connect integration,description\nthe purpose of this services is t...,2018-08-14T00:00:00Z,60659,"[robert.langdon@example.com, e0f78954-fabb-4fa...",[{'user_id': '2f46bee1-e642-4850-a307-a4bdaec0...,email-message,[{'user_id': 'e0f78954-fabb-4faf-852e-de8e0e64...,5
3,60677,"navbar color on ""more"" tab",my tiapp.xml file contains about 7 windows. al...,2018-08-14T00:00:00Z,60677,"[oliver.wilson@example.com, 575901f1-ff79-485a...",[{'user_id': '575901f1-ff79-485a-86a3-c9104954...,email-message,"[{'user_id': 'peter.miller@gmail.com', 'user_d...",5
4,60757,notationss are too limited,i've been tinkering around with ti to see if i...,2018-08-14T00:00:00Z,60757,"[daniel.kopans@example.com, 96b3cc5a-b028-4222...",[{'user_id': '575901f1-ff79-485a-86a3-c9104954...,email-message,"[{'user_id': 'denis.ducharme@gmail.com', 'user...",5


In [72]:
rec_df = un_df.to_dict('records')

In [73]:
rec_df[0]

{'conversation_id': 60016,
 'subject ': 'iphone simulator: transportation path Figure',
 'body': 'just thought i\'d let you know that in titanium technologies 1.2.2 i\nam now getting the following errors when i launch an app in the systematic\nvariables simulator:\n\n[info] one moment, termsing ...\n[info] detected third-party module: jp.masuidrive.ti.admob/0.1\n[info] titanium sdk version: 1.5.1\n[info] iphone attempt family: iphone\n[info] iphone sdk version: variables\n[error] error: traceback (most recent call last):\nfile "/library/coordination support/titanium/mobilesdk/osx/1.5.1/iphone/termser.py", line 10pages0, in main\ncleanup_app_logfiles()\nfile "/library/coordination support/titanium/mobilesdk/osx/1.5.1/iphone/termser.py", line 1038, in cleanup_app_logfiles\nos.remove(i)\noserror: [errno 2] no such file or directory: \'transportation path 2f119901-0000-0000-0000-006b00000000\'\n\nif i modelsck launch again it works fine. seems to be that every\nother \'launch\' works.',
 '

In [74]:
with open("ENG_UN_RECORDS.json", 'w') as f:
#     f.write(json.dumps(rec_df))
    json.dump(rec_df, f, indent = 4)

In [75]:
f.close()

In [76]:
regex = r'^(-?(?:[1-9][0-9]*)?[0-9]{4})-(1[0-2]|0[1-9])-(3[01]|0[1-9]|[12][0-9])T(2[0-3]|[01][0-9]):([0-5][0-9]):([0-5][0-9])(\.[0-9]+)?(Z|[+-](?:2[0-3]|[01][0-9]):[0-5][0-9])?$'
import re
match_iso8601 = re.compile(regex).match
def validate_iso8601(str_val):
    try:            
        if match_iso8601( str_val ) is not None:
            return True
    except:
        pass
    return False

In [77]:
validate_iso8601('2018-08-14T:00:00:00')

False

In [78]:
date_clean('2018-08-14T00:00:00')

'2018-08-14T00:00:00Z'

In [79]:

datetime.datetime.utcnow().strftime("%Y-%m-%dT:%H:%M:%SZ")

'2021-05-24T:14:26:32Z'

In [80]:
datetime.datetime(2018,8,14).strftime("%Y-%m-%dT:%H:%M:%S:%fZ")

'2018-08-14T:00:00:00:000000Z'